In [1]:
# 本地
input_dir,project_dir = r"G:\\Browser\\GAIDC_preliminary_A\\", r"G:\\Browser\\GAIDC_preliminary_A\\"
# 和鲸
# input_dir,project_dir = r"/home/mw/input/rcxx20231783/", r"/home/mw/project/"
# # AIstudio
# input_dir,project_dir = r"/home/aistudio/data/data187170/", r"/home/aistudio/work/"

test_dir = input_dir + 'preliminary_A.csv'
train_dir = input_dir + 'train.csv'
prediction_dir = project_dir + 'prediction.csv'
pickle_dir = project_dir + 'df.pickle'

# 全局超参
n_jobs = 8 # 核心数
random_state = 123 # 随机数种子
n_estimators = 10000 # 迭代次数
early_stopping_rounds = int(n_estimators/10)

frac = 1 # 读取数据的比例（用于debug）

In [2]:
drop_list = ['date_id']
y_cols = 'y'

In [4]:
import sklearn
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import explained_variance_score
from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
from sklearn. model_selection import train_test_split
from tqdm import tqdm
import datetime
import time
import re
import pickle
import math



from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
import lightgbm as lgb
from lightgbm import LGBMClassifier
import gc
from sklearn.metrics import roc_auc_score
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 300)
import warnings
warnings.filterwarnings("ignore")

In [5]:
# df = pd.read_csv("/home/mw/input/pre8881/train.csv")#训练集
# df = pd.DataFrame(df.values[0:10000],columns=df.columns) #测试集
df_test = pd.read_csv(test_dir)
df_train = pd.read_csv(train_dir)
# df_train.dropna(thresh = len(df_train.columns) - df_train.isnull().sum(axis=1).max() + 1,inplace=True)#dropna空最多的TopN
df = df_train.append(df_test, sort=False)#.sample(frac=0.1, random_state=random_state)# 合并测试集与训练集
# del df_train,df_test
gc.collect()

4

## 数据预处理


In [6]:
le = LabelEncoder()
for f in tqdm(list(df.drop(columns=['id']).select_dtypes(include='object'))):
    '''
    LabelEncoding (for object)
    '''
    try:
        if f=="id": # 保留id
            continue
        df[f] = df[f].astype('float')# 将数字转成数值型数据
    except:
        df[f] = le.fit_transform(df[f].astype('str')).astype('int')# 字符串label化

100%|██████████| 3/3 [00:00<00:00, 150.01it/s]


## 构造衍生业务特征

In [7]:
# 数量编码
groupby_list = ['transformers', 'month','date_id', 'is_weekend', 'time', 'L1', 'L2', 'L3', 'L4', 'L5', 'L6']
for name in tqdm(groupby_list):
    # 加上同类数属性列
    df_temp = df.groupby(name).size().reset_index()#重置索引
    df_temp.columns = [name] + ["num_in_"+name]
    df = df.merge(df_temp, how='left')  # 左合并   

100%|██████████| 11/11 [00:00<00:00, 57.70it/s]


In [8]:
L_list = ['L1', 'L2', 'L3', 'L4', 'L5', 'L6']
L_norm_list = [i+"_norm" for i in L_list]
max_min_scaler = lambda x : (x-x.min())/(x.max()-x.min())
for i in L_list: # 将L1-6进行min-max归一化
    df[f'{i}_norm'] = df[[i]].apply(max_min_scaler).values
df["L_norm_sum"] = df[L_norm_list].sum(axis=1) # 归一化总和
df["L_sum"] = df[L_list].sum(axis=1) # 总和
df['L_norm_std']  = df[L_norm_list].std(axis=1) # 方差
df['L_std']  = df[L_list].std(axis=1) # 方差

In [9]:
# 猜测复杂L1-L6是否大于0可能会有影响
ge0 = lambda x: 1 if x>=0 else 0
L_ge0_list = [f"is_{i}_ge0" for i in L_list]
for i in L_list:
    df[f'is_{i}_ge0'] = df[[i]].applymap(ge0).values
df["num_L_ge0"] = df[L_ge0_list].sum(axis=1)
df["is_L_all_ge0"] = df["is_L1_ge0"] * df["is_L2_ge0"] * df["is_L3_ge0"] * df["is_L4_ge0"] * df["is_L5_ge0"] * df["is_L6_ge0"]

# 特征构建

In [10]:
# 统计特征用到的函数
def stat(df, df_merge, cols, agg): # 每次只计算一列cols
    # 测试集df_merge关于训练集df的cols列agg统计特征
    group = df.groupby(cols).agg(agg) #算出cols属性在df中的agg属性
    columns = []
    for on, methods in agg.items(): 
        #items为y_cols、mean;此处为字典变量————'y_cols':['mean','std']
        for method in methods: # methods也可能有多个
            if str(type(method)) == '<class \'function\'>': # 匿名函数会以地址命名，造成列名不同，所以应该更名
                method = 'mode'
            columns.append('{}_{}_{}'.format('_'.join(cols), on, method)) # 用_连接，规定分组结果group的属性名
    group.columns = columns
    group.reset_index(inplace=True)
    df_merge = df_merge.merge(group, on=cols, how='left') #以cols左连接，仅与测试集拼接
    del (group)
    gc.collect()
    return df_merge


def statis_feat(df_know, df_unknow,A_list,target_B, methods_dict):
    # 计算A关于B的统计特征：
    # df_know训练集,df_unknow验证集,A_list需计算的特征集合,target_B目标标签,methods_dict编码方式
    for f in tqdm(A_list,desc='%-30s'%target_B,leave=False):
        df_unknow = stat(df_know, df_unknow, [f], {target_B: methods_dict}) # 
    return df_unknow


def multi_groupby(df_fold_train,df_fold_val,multi_groupby_cols,target_B, methods_dict):
    ''' 
    计算测试集df_fold_val关于训练集df_fold_train的multi_groupby_cols组合列methods_dict统计特征。
    注意！如果df_fold_val在df_fold_train中找不到相应的值，则会置为nan
    '''
    multi_groupby_df1 = df_fold_train.groupby(multi_groupby_cols).agg({target_B: methods_dict})
    multi_groupby_df1.columns=["_".join(multi_groupby_cols+[target_B,"_".join(methods_dict)])]
    df_fold_val = df_fold_val.merge(multi_groupby_df1,how='left',left_on=multi_groupby_cols,right_index=True)
    return df_fold_val


In [11]:
# 组合特征目标编码
# multi_groupby_cols = [
# 	   ['month','is_weekend'],['transformers','month']]
def generate_multi_groupby_cols(target:str,rank:int)->list:
    multi_groupby_cols =  []
    feature_list = ['transformers', 'month', 'is_weekend', 'time', 'L1', 'L2', 'L3', 'L4', 'L5', 'L6']
    if target!="y":
        feature_list.remove(target)

    if rank>=1:
        # 一阶
        for i in range(len(feature_list)):
            multi_groupby_cols.append([feature_list[i]])
    if rank>=2:  
        # 二阶
        for i in range(len(feature_list)):
            for j in range(i+1,len(feature_list)):
                multi_groupby_cols.append([feature_list[i],feature_list[j]])
    if rank>=3:
        # 三阶
        for i in range(len(feature_list)-2):
            for j in range(i+1,len(feature_list)-1):
                for h in range(j+1,len(feature_list)):
                    multi_groupby_cols.append([feature_list[i],feature_list[j],feature_list[h]])
    if rank>=4:
        # 四阶
        for x in range(len(feature_list)-3):
            for i in range(x+1,len(feature_list)-2):
                for j in range(i+1,len(feature_list)-1):
                    for h in range(j+1,len(feature_list)):
                        multi_groupby_cols.append([feature_list[x],feature_list[i],feature_list[j],feature_list[h]]) 
    if rank>=5:
        # 五阶
        for x in range(len(feature_list)-4):
            for y in range(x+1,len(feature_list)-3):
                for i in range(y+1,len(feature_list)-2):
                    for j in range(i+1,len(feature_list)-1):
                        for h in range(j+1,len(feature_list)):
                            multi_groupby_cols.append([feature_list[x],feature_list[y],feature_list[i],feature_list[j],feature_list[h]])              
    return multi_groupby_cols
y_multi_groupby_cols = generate_multi_groupby_cols('y',4)
L1_multi_groupby_cols = generate_multi_groupby_cols('L1',2)
L2_multi_groupby_cols = generate_multi_groupby_cols('L2',2)
L3_multi_groupby_cols = generate_multi_groupby_cols('L3',2)
L4_multi_groupby_cols = generate_multi_groupby_cols('L4',2)
L5_multi_groupby_cols = generate_multi_groupby_cols('L5',2)
L6_multi_groupby_cols = generate_multi_groupby_cols('L6',2)
y_multi_groupby_cols = [['transformers'], ['month'], ['is_weekend'], ['time']]

In [12]:
df_train = df[~df[y_cols].isnull()] # 重新分离出训练集和测试集
df_train = df_train.reset_index(drop=True)
df_test = df[df[y_cols].isnull()]

df_train_stas_feat = None
# 为了防止过拟合，采取KFold法统计train训练集中的特征
kfold = KFold(n_splits=5, random_state=random_state, shuffle=True)
for fold_id, (trn_idx, val_idx) in enumerate(kfold.split(df_train, df_train[y_cols])):
    print(f'\nFold_{fold_id+1} Featuring ==========={time.asctime(time.localtime(time.time()))}\n')
    df_fold_train,df_fold_val = df_train.iloc[trn_idx],df_train.iloc[val_idx] # 划分训练集与验证集
    
    # 计算验证集关于训练集的单/多列组合统计特征
    for cols in tqdm(y_multi_groupby_cols,desc='%-30s'%'y_multi_groupby_cols',leave=False): 
        df_fold_val = multi_groupby(df_fold_train, df_fold_val, cols,y_cols, ['mean'])

    # for cols in tqdm(L1_multi_groupby_cols,desc='%-30s'%'L1_multi_groupby_cols',leave=False): 
    #     df_fold_val = multi_groupby(df_fold_train, df_fold_val, cols,'L1', ['mean'])
    # for cols in tqdm(L2_multi_groupby_cols,desc='%-30s'%'L2_multi_groupby_cols',leave=False): 
    #     df_fold_val = multi_groupby(df_fold_train, df_fold_val, cols,'L2', ['mean'])
    # for cols in tqdm(L3_multi_groupby_cols,desc='%-30s'%'L3_multi_groupby_cols',leave=False): 
    #     df_fold_val = multi_groupby(df_fold_train, df_fold_val, cols,'L3', ['mean'])
    # for cols in tqdm(L4_multi_groupby_cols,desc='%-30s'%'L4_multi_groupby_cols',leave=False): 
    #     df_fold_val = multi_groupby(df_fold_train, df_fold_val, cols,'L4', ['mean'])
    # for cols in tqdm(L5_multi_groupby_cols,desc='%-30s'%'L5_multi_groupby_cols',leave=False): 
    #     df_fold_val = multi_groupby(df_fold_train, df_fold_val, cols,'L5', ['mean'])
    # for cols in tqdm(L6_multi_groupby_cols,desc='%-30s'%'L6_multi_groupby_cols',leave=False): 
    #     df_fold_val = multi_groupby(df_fold_train, df_fold_val, cols,'L6', ['mean'])

    df_train_stas_feat = pd.concat([df_train_stas_feat, df_fold_val], axis=0)# 五折之后拼成一个完整的stas_feat训练集
    # del df_fold_train,df_fold_val
    gc.collect()

# 计算测试集关于训练集的统计特征
for cols in tqdm(y_multi_groupby_cols,desc='%-30s'%'y_multi_groupby_cols',leave=False):
    df_test = multi_groupby(df_train, df_test, cols,y_cols, ['mean'])

# for cols in tqdm(L1_multi_groupby_cols,desc='%-30s'%'L1_multi_groupby_cols',leave=False):
#     df_test = multi_groupby(df_train, df_test, cols,'L1', ['mean'])
# for cols in tqdm(L2_multi_groupby_cols,desc='%-30s'%'L2_multi_groupby_cols',leave=False):
#     df_test = multi_groupby(df_train, df_test, cols,'L2', ['mean'])
# for cols in tqdm(L3_multi_groupby_cols,desc='%-30s'%'L3_multi_groupby_cols',leave=False):
#     df_test = multi_groupby(df_train, df_test, cols,'L3', ['mean'])
# for cols in tqdm(L4_multi_groupby_cols,desc='%-30s'%'L4_multi_groupby_cols',leave=False):
#     df_test = multi_groupby(df_train, df_test, cols,'L4', ['mean'])
# for cols in tqdm(L5_multi_groupby_cols,desc='%-30s'%'L5_multi_groupby_cols',leave=False):
#     df_test = multi_groupby(df_train, df_test, cols,'L5', ['mean'])
# for cols in tqdm(L6_multi_groupby_cols,desc='%-30s'%'L6_multi_groupby_cols',leave=False):
#     df_test = multi_groupby(df_train, df_test, cols,'L6', ['mean'])

df = pd.concat([df_train_stas_feat, df_test], axis=0) # 将分别完成的训练集和测试集的stas_feat合并成最终的df
df.info()


Fold_1 Featuring ===========Sun Mar  5 19:10:47 2023




Fold_2 Featuring ===========Sun Mar  5 19:10:47 2023




Fold_3 Featuring ===========Sun Mar  5 19:10:47 2023




Fold_4 Featuring ===========Sun Mar  5 19:10:47 2023




Fold_5 Featuring ===========Sun Mar  5 19:10:47 2023



<class 'pandas.core.frame.DataFrame'>
Int64Index: 48000 entries, 1 to 47999
Data columns (total 46 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   48000 non-null  object 
 1   transformers         48000 non-null  int32  
 2   date_id              48000 non-null  int32  
 3   month                48000 non-null  int64  
 4   is_weekend           48000 non-null  int64  
 5   time                 48000 non-null  int32  
 6   L1                   48000 non-null  float64
 7   L2                   48000 non-null  float64
 8   L3                   48000 non-null  float64
 9   L4                   48000 non-null  float64
 10  L5                   48000 non-null  float64
 11  L6                   48000 non-null  float64
 12  y                    38400 non-null  float64
 13  num_in_transformers  48000 non-null  int64  
 14  num_in_month         48000 non-null  int64  
 15  num_in_date_id       48000 non-null 

In [13]:
for i in df.columns.values:
    # 填充空值，y除外
    if i != "y":
        df[i].fillna(-100,inplace=True)

for i,j in zip(df.isna().sum().index, df.isna().sum()):
    # 无输出则无空值
    if(j!=0):
        print(i,j)

# 分割train和test
df_test = df[df[y_cols].isnull()].drop(columns=drop_list)
df_train = df[df[y_cols].notnull()].drop(columns=drop_list)

y 9600


In [14]:
# 使用 `pickle.dump()` 函数将变量保存到文件中
# with open(pickle_dir, 'wb') as f:
#     pickle.dump(df, f)

# with open(pickle_dir, 'rb') as f:
#     df = pickle.load(f)

# 数据训练

In [15]:
# 新建一个dataframe，总行数为原df的1/48，将原df的所有feat都整合成长48的list
column_names = list(df.columns.values) #.remove("id")
# column_names.remove("id")
df_48 = pd.DataFrame(columns=column_names)
# test_id_list = df_test["id"].values.tolist() # 存储测试集id，便于分隔
df_48.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 46 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id                   0 non-null      object
 1   transformers         0 non-null      object
 2   date_id              0 non-null      object
 3   month                0 non-null      object
 4   is_weekend           0 non-null      object
 5   time                 0 non-null      object
 6   L1                   0 non-null      object
 7   L2                   0 non-null      object
 8   L3                   0 non-null      object
 9   L4                   0 non-null      object
 10  L5                   0 non-null      object
 11  L6                   0 non-null      object
 12  y                    0 non-null      object
 13  num_in_transformers  0 non-null      object
 14  num_in_month         0 non-null      object
 15  num_in_date_id       0 non-null      object
 16  num_in_is_weekend    0 no

In [16]:
feat_name = df_48.columns.values
# feat_name = column_names
# 把M1和M2分开
for transformers in df["transformers"].value_counts().index.values:
    date_id_list = df[df["transformers"] == transformers]["date_id"].value_counts().index.values
    for date_id in tqdm(date_id_list):
        tmp = df.query(f' transformers=={transformers} and date_id=={date_id} ')
        data = {i: tmp[i].values for i in feat_name}
        # data = {
        #     "y": df_list["y"].values,
        #     "L1": df_list["L1"].values,
        #     "L2": df_list["L2"].values,
        #     "L3": df_list["L3"].values,
        #     }
        df_48 = df_48.append(data, ignore_index=True)
# df_48[["id","y"]]
df_48.info()

100%|██████████| 483/483 [00:07<00:00, 62.53it/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 46 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id                   1000 non-null   object
 1   transformers         1000 non-null   object
 2   date_id              1000 non-null   object
 3   month                1000 non-null   object
 4   is_weekend           1000 non-null   object
 5   time                 1000 non-null   object
 6   L1                   1000 non-null   object
 7   L2                   1000 non-null   object
 8   L3                   1000 non-null   object
 9   L4                   1000 non-null   object
 10  L5                   1000 non-null   object
 11  L6                   1000 non-null   object
 12  y                    1000 non-null   object
 13  num_in_transformers  1000 non-null   object
 14  num_in_month         1000 non-null   object
 15  num_in_date_id       1000 non-null   object
 16  num_in_

In [17]:
# 区分train和test
test_no_list = []
train_no_list = [i for i in range(1000)]
for i,j in enumerate(df_48["y"]):
    if j.tolist() != j.tolist(): # 即为nan
        # print(j)
        test_no_list.append(i)
        train_no_list.remove(i)

len(test_no_list), len(train_no_list)

(200, 800)

In [18]:
# 把y换到最后一列
exc_y_list = df_48.columns.values.tolist()
exc_y_list.remove('y')
exc_y_list
y_value = df_48["y"].values
# df_48 = df_48[exc_y_list].copy()
df_48 = df_48.drop(columns=["y"])
df_48["y"] = y_value
df_48.info()
# test_no_list

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 46 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id                   1000 non-null   object
 1   transformers         1000 non-null   object
 2   date_id              1000 non-null   object
 3   month                1000 non-null   object
 4   is_weekend           1000 non-null   object
 5   time                 1000 non-null   object
 6   L1                   1000 non-null   object
 7   L2                   1000 non-null   object
 8   L3                   1000 non-null   object
 9   L4                   1000 non-null   object
 10  L5                   1000 non-null   object
 11  L6                   1000 non-null   object
 12  num_in_transformers  1000 non-null   object
 13  num_in_month         1000 non-null   object
 14  num_in_date_id       1000 non-null   object
 15  num_in_is_weekend    1000 non-null   object
 16  num_in_

In [19]:
# 数据框转换为numpy数组
df_train_np = df_48.iloc[train_no_list].drop(columns="id").to_numpy() # 抽取出train

diff = len(["id","y"])
# 提取出list
df_train_list = []
for i in range(len(df_48.columns.values) - diff): # 去除y和id所以减2
    df_train_list.append(df_train_np[:, i])
label = df_train_np[:, -1] # 取y

# 将特征A，特征B和标签转换为三维张量。将数值类型从list转换为array
x_train = np.array(
    [
        [df_train_list[i][j] for i in range(len(df_48.columns.values) - diff)] 
        for j in range(len(df_train_list[0]))
    ]
    )
# 优先外层的for
# Dense需要T转置,可能需要在label[i]外再加一层[]
y_train = np.array([[label[i]] for i in range(len(label))]) #.T 

# ================测试集================

df_test_np = df_48.iloc[test_no_list].drop(columns="id").to_numpy() # 抽取出test
# 提取出list
df_test_list = []
for i in range(len(df_48.columns.values) - diff):
    df_test_list.append(df_test_np[:, i])

x_test = np.array(
    [
        [df_test_list[i][j] for i in range(len(df_48.columns.values) - diff)] 
        for j in range(len(df_test_list[0]))
    ]
    )
x_train.shape, y_train.shape

((800, 44, 48), (800, 1, 48))

In [20]:
import time
from sklearn.model_selection import KFold
from tensorflow.keras.layers import LSTM, Dense, Dropout, Conv1D, MaxPooling1D, Flatten, Input, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model


# 五折交叉验证
folds = KFold(n_splits=5, shuffle=True, random_state=2019)
oof = np.zeros([len(x_train), 48])
predictions = np.zeros([len(x_test), 48])

input_shape=tuple([x_train.shape[i] for i in range(1,len(x_train.shape))]) # 输入数据维度（忽略条数

for fold_, (trn_idx, val_idx) in enumerate(
    folds.split(range(len(x_train)), y_train)
    ):
    # KFold五折交叉
    print(f'\nFold_{fold_+1} Featuring ==========={time.asctime(time.localtime(time.time()))}\n')
    X_tra, X_val = x_train[trn_idx], x_train[val_idx]
    y_tra, y_val = y_train[trn_idx], y_train[val_idx]
    
    model = Sequential()
    
    '''LSTM'''
    model.add(LSTM(320, input_shape=(x_train.shape[1], 48))) # input_shape（特征数量，特征值维数）
    model.add(Dropout(0.2))
    model.add(Dense(48))

    '''MLP'''
    # model.add(Dense(320, input_shape=input_shape))
    # model.add(BatchNormalization())
    # model.add(Dense(128))
    # model.add(Dropout(0.2))
    # model.add(Dense(64))
    # model.add(Dense(48)) # , activation='linear'

    '''CNN'''
    # 输入数据
    # input_tensor = Input(shape=input_shape)
    # # 卷积层
    # x = Conv1D(filters=3, kernel_size=3, activation='relu')(input_tensor)
    # x = MaxPooling1D(pool_size=2)(x)
    # # 全连接层
    # x = Flatten()(x)
    # x = Dense(units=48)(x)
    # # 模型定义
    # model = Model(input_tensor, x)

    # 编译模型
    model.compile(loss='mean_squared_error', optimizer='adam',metrics=['mae'])

    # 训练模型
    # x_train是特征A和特征B的训练数据，y_train是标签的训练数据
    # x_train和y_train需要转换成三维张量
    model.fit(X_tra, y_tra, epochs=2000, 
        batch_size=8, 
        validation_data=(X_val, y_val), 
        callbacks=[EarlyStopping(monitor='loss', patience=10)])

    tmp = model.predict(X_val)
    print(tmp.shape)
    print(tmp)
    oof[val_idx] = tmp

    tmp = model.predict(x_test)
    print(tmp)
    predictions += tmp / folds.n_splits

    # print(predictions)
    del model



Fold_1 Featuring ===========Sun Mar  5 19:11:08 2023

Epoch 1/2000
80/80 [==============================] - 5s 12ms/step - loss: 306.8897 - mae: 13.4202 - val_loss: 152.8471 - val_mae: 9.5010
Epoch 2/2000
80/80 [==============================] - 1s 8ms/step - loss: 154.8224 - mae: 10.0285 - val_loss: 137.5700 - val_mae: 9.6111
Epoch 3/2000
80/80 [==============================] - 1s 8ms/step - loss: 150.9140 - mae: 10.2858 - val_loss: 136.8925 - val_mae: 9.5072
Epoch 4/2000
80/80 [==============================] - 1s 8ms/step - loss: 150.3680 - mae: 10.2241 - val_loss: 137.0502 - val_mae: 9.5452
Epoch 5/2000
80/80 [==============================] - 1s 9ms/step - loss: 150.2576 - mae: 10.2336 - val_loss: 137.8802 - val_mae: 9.6480
Epoch 6/2000
80/80 [==============================] - 1s 8ms/step - loss: 150.4595 - mae: 10.2189 - val_loss: 137.6074 - val_mae: 9.6188
Epoch 7/2000
80/80 [==============================] - 1s 8ms/step - loss: 150.5839 - mae: 10.3031 - val_loss: 137.0701 - v

In [31]:
y_train = y_train.reshape((800,48)),#  oof.shape, predictions.shape
y_train.shape()

AttributeError: 'tuple' object has no attribute 'reshape'

In [32]:
# from sklearn.metrics import mean_squared_error, mean_absolute_error
# score = mean_absolute_error(y_train, oof)
# score, predictions.shape

ValueError: Found input variables with inconsistent numbers of samples: [1, 800]

In [33]:
df_test_48 = df_48.iloc[test_no_list]
for i,j in enumerate(df_test_48["y"]):
    df_test_48.iloc[i,-1] = predictions[i]
# df_test_48[["id","y"]]

In [34]:
# df_test_48[["id",]].values[0].tolist()
# arr.flatten()
test_id_list = np.concatenate(df_test_48["id"].values)
test_y_list = np.concatenate(df_test_48["y"].values)
df_test[["id","y"]]

for i,j in tqdm(zip(test_id_list, test_y_list)):
    df_test.loc[df_test["id"]==i,"y"] = j

9600it [00:04, 2052.04it/s]


In [35]:
df_test["y"]

38400    18.168720
38401    17.866105
38402    18.247434
38403    19.049457
38404    19.995107
           ...    
47995    20.606771
47996    21.158372
47997    21.031034
47998    20.341500
47999    19.428096
Name: y, Length: 9600, dtype: float64

In [36]:
# df_test[["id","y"]].to_csv(f'lstm_{int(100*score)}.csv', index=False)
df_test[["id","y"]].to_csv(f'lstm.csv', index=False)